In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import keras
import cv2
import pickle
import sklearn
import random

from sklearn.metrics import auc, f1_score, roc_curve, recall_score, precision_score, accuracy_score, confusion_matrix
from sklearn import metrics
from google.colab import files
from keras.preprocessing import image
# from keras.layers.serialization import activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import *
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.models import load_model
from keras import backend as K
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
base_dir = '/content/drive/MyDrive/TIFF PET-CTWholeDataset'

training_set_dir = os.path.join(base_dir, 'trainingSet')
test_set_dir = os.path.join(base_dir, 'testSet')

train_lungCancer_dir = os.path.join(training_set_dir, 'lungCancer')
train_normal_dir = os.path.join(training_set_dir, 'normal')
test_lungCancer_dir = os.path.join(test_set_dir, 'lungCancer')
test_normal_dir = os.path.join(test_set_dir, 'normal')

In [18]:
print('total training lungCancer images:', len(os.listdir(train_lungCancer_dir)))
print('total training normal images:', len(os.listdir(train_normal_dir)))
print('total test lungCancer images:', len(os.listdir(test_lungCancer_dir)))
print('total test normal images:', len(os.listdir(test_normal_dir)))

total training lungCancer images: 719
total training normal images: 726
total test lungCancer images: 199
total test normal images: 207


In [19]:
def check_data_leakage(directory1, directory2):
    filenames_dir1 = set(os.listdir(directory1))
    filenames_dir2 = set(os.listdir(directory2))

    common_filenames = filenames_dir1.intersection(filenames_dir2)

    if len(common_filenames) > 0:
        print("Data leakage detected!")
        print("Common image filenames between the directories:", common_filenames)
    else:
        print("No data leakage detected.")

In [20]:
check_data_leakage(train_lungCancer_dir, test_lungCancer_dir)
check_data_leakage(train_lungCancer_dir, test_normal_dir)
check_data_leakage(train_normal_dir, test_normal_dir)
check_data_leakage(train_normal_dir, test_lungCancer_dir)

No data leakage detected.
No data leakage detected.
No data leakage detected.
No data leakage detected.


In [21]:
BATCH_SIZE= 80
image_height, image_width = 245, 457

train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                  rotation_range= 15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  zoom_range=0.1,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        training_set_dir,
        target_size=(image_height, image_width),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle = True)


test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
        test_set_dir,
        target_size=(image_height, image_width),
        batch_size=10,
        class_mode='binary')

Found 1445 images belonging to 2 classes.
Found 406 images belonging to 2 classes.


In [22]:
pre_trained_model = InceptionV3(include_top=False,
                                        weights= 'imagenet',
                                        input_shape = (image_height, image_width, 3))

for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 245, 457, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 122, 228, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 122, 228, 32)         96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_95 (Activation)  (None, 122, 228, 32)         0         ['batch_normalizati

In [23]:
last_layer = pre_trained_model.get_layer('mixed10')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dense(1)(x)
x = layers.Activation(tf.nn.sigmoid)(x)

model = Model(pre_trained_model.input, x)

last layer output shape:  (None, 6, 12, 2048)


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 245, 457, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 122, 228, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 122, 228, 32)         96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_95 (Activation)  (None, 122, 228, 32)         0         ['batch_normalization_94

In [26]:
# savingPath = os.makedirs('/content/drive/MyDrive/InceptionV3Weights/1')
checkpoint_path = '/content/drive/MyDrive/InceptionV3Weights/1'
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, save_best_only=True, verbose= 1)

/content/drive/MyDrive/InceptionV3Weights/1


In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001), loss= 'binary_crossentropy',
              metrics= [tf.keras.metrics.BinaryAccuracy(name="Binary_accuracy"),
                        tf.keras.metrics.Precision(name='Precision'),
                        tf.keras.metrics.Recall(name='Recall'),
                        tf.keras.metrics.TruePositives(name='TP'),
                        tf.keras.metrics.TrueNegatives(name='TN'),
                        tf.keras.metrics.FalseNegatives(name='FN'),
                        tf.keras.metrics.FalsePositives(name='FP'),
                        tf.keras.metrics.AUC(name='AUC')])

In [ ]:
history = model.fit(
            train_generator,
            validation_data = test_generator,
            epochs = 40,
            verbose = 1,
            callbacks = [cp_callback])

In [29]:
model2 = Model(pre_trained_model.input, x)

In [30]:
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001), loss= 'binary_crossentropy',
               metrics= [tf.keras.metrics.BinaryAccuracy(name="Binary_accuracy"),
                        tf.keras.metrics.Precision(name='Precision'),
                        tf.keras.metrics.Recall(name='Recall'),
                        tf.keras.metrics.TruePositives(name='TP'),
                        tf.keras.metrics.TrueNegatives(name='TN'),
                        tf.keras.metrics.FalseNegatives(name='FN'),
                        tf.keras.metrics.FalsePositives(name='FP'),
                        tf.keras.metrics.AUC(name='AUC')])

In [31]:
model2.load_weights(checkpoint_path)

In [32]:
model2.evaluate(test_generator)

41/41 [==============================] - 97s 2s/step - loss: 0.4650 - Binary_accuracy: 0.8571 - Precision: 0.8984 - Recall: 0.8116 - TP: 168.0000 - TN: 180.0000 - FN: 39.0000 - FP: 19.0000 - AUC: 0.9112


[0.4650360643863678,
 0.8571428656578064,
 0.8983957171440125,
 0.8115941882133484,
 168.0,
 180.0,
 39.0,
 19.0,
 0.9111863970756531]

In [ ]:
base_dir = '/content/drive/MyDrive/TIFF PET-CTWholeDataset'

training_set_dir = os.path.join(base_dir, 'trainingSet')
test_set_dir = os.path.join(base_dir, 'testSet')

train_lungCancer_dir = os.path.join(training_set_dir, 'lungCancer')
train_normal_dir = os.path.join(training_set_dir, 'normal')
test_lungCancer_dir = os.path.join(test_set_dir, 'lungCancer')
test_normal_dir = os.path.join(test_set_dir, 'normal')

print(test_normal_dir)
print(test_lungCancer_dir)

/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/normal
/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/lungCancer


In [ ]:
fileNames2 = os.listdir(test_lungCancer_dir)
filePaths2 = []

for i in fileNames2:
    path = '/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/lungCancer/'
    new_path = path + i
    filePaths2.append(new_path)


my_y_pred2 = []


for fn in filePaths2:
    img = tf.keras.utils.load_img(fn, target_size=(245, 457))
    x = tf.keras.utils.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model2.predict(images, batch_size=10)
    my_y_pred2.append(classes)
    print(classes[0])
    if classes[0]>0.5:
        print(fn + " lunCancer does not exist in this slice")
    else:
        print(fn + " lungCancer exists in this slice")


y_pred2 = np.array(my_y_pred2)
y_pred2.resize(1,len(my_y_pred2))
y_pred2

1/1 [==============================] - 1s 1s/step
[0.00088693]
/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/lungCancer/0000156 (10).tif lungCancer exists in this slice
1/1 [==============================] - 0s 154ms/step
[0.00083663]
/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/lungCancer/0000156 (1).tif lungCancer exists in this slice
1/1 [==============================] - 0s 164ms/step
[0.00118003]
/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/lungCancer/0000156 (11).tif lungCancer exists in this slice
1/1 [==============================] - 0s 166ms/step
[0.00324874]
/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/lungCancer/0000156 (13).tif lungCancer exists in this slice
1/1 [==============================] - 0s 139ms/step
[0.00570195]
/content/drive/MyDrive/TIFF PET-CTWholeDataset/testSet/lungCancer/0000156 (12).tif lungCancer exists in this slice
1/1 [==============================] - 0s 150ms/step
[0.00620392]
/content/drive/MyDrive/TIFF PET-C

In [33]:
base_dir3 = '/content/drive/MyDrive/2ExternalDataSet'

In [34]:
fileNames2 = os.listdir(base_dir3)
filePaths2 = []

for i in fileNames2:
    path = '/content/drive/MyDrive/2ExternalDataSet/'
    new_path = path + i
    filePaths2.append(new_path)


my_y_pred2 = []


for fn in filePaths2:
    img = tf.keras.utils.load_img(fn, target_size=(245, 457))
    x = tf.keras.utils.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model2.predict(images, batch_size=10)
    my_y_pred2.append(classes)
    print(classes[0])
    if classes[0]>0.5:
        print(fn + " lunCancer does not exist in this slice")
    else:
        print(fn + " lungCancer exists in this slice")


y_pred2 = np.array(my_y_pred2)
y_pred2.resize(1,len(my_y_pred2))
y_pred2

1/1 [==============================] - 2s 2s/step
[0.00569852]
/content/drive/MyDrive/2ExternalDataSet/197 (77).tif lungCancer exists in this slice
1/1 [==============================] - 0s 455ms/step
[0.02296]
/content/drive/MyDrive/2ExternalDataSet/197 (79).tif lungCancer exists in this slice
1/1 [==============================] - 0s 481ms/step
[0.01149152]
/content/drive/MyDrive/2ExternalDataSet/197 (78).tif lungCancer exists in this slice
1/1 [==============================] - 0s 493ms/step
[0.00445311]
/content/drive/MyDrive/2ExternalDataSet/197 (81).tif lungCancer exists in this slice
1/1 [==============================] - 1s 502ms/step
[0.01455113]
/content/drive/MyDrive/2ExternalDataSet/197 (80).tif lungCancer exists in this slice
1/1 [==============================] - 0s 499ms/step
[0.00341729]
/content/drive/MyDrive/2ExternalDataSet/197 (82).tif lungCancer exists in this slice
1/1 [==============================] - 0s 439ms/step
[0.01169947]
/content/drive/MyDrive/2ExternalDa

array([[0.00569852, 0.02296   , 0.01149152, ..., 0.01588395, 0.00867003,
        0.00656957]], dtype=float32)

In [35]:
binary_predictions = (y_pred2.flatten() > 0.5).astype(int)
true_labels = np.zeros_like(binary_predictions)
accuracy = accuracy_score(true_labels, binary_predictions)
print(f"Accuracy: {accuracy:.4f}")
sensitivity = recall_score(true_labels, binary_predictions, pos_label=0)
print(f"Sensitivity (Recall): {sensitivity:.4f}")

Accuracy: 0.8022
Sensitivity (Recall): 0.8022


In [ ]:
import os
from collections import defaultdict


base_dir3 = '/content/drive/MyDrive/2ExternalDataSet'


fileNames2 = os.listdir(base_dir3)
filePaths2 = []

for i in fileNames2:
    path = '/content/drive/MyDrive/2ExternalDataSet/'
    new_path = path + i
    filePaths2.append(new_path)


patient_predictions = defaultdict(list)


for fn in filePaths2:
    img = tf.keras.utils.load_img(fn, target_size=(245, 457))
    x = tf.keras.utils.img_to_array(img)
    x /= 255  
    x = np.expand_dims(x, axis=0)  
    images = np.vstack([x])  

    
    classes = model2.predict(images, batch_size=10)
    prediction = classes[0][0]  

    
    patient_id = os.path.splitext(fn)[0].split('(')[0].strip()

    
    patient_predictions[patient_id].append(prediction)


final_patient_predictions = []
true_labels = []  

In [299]:
for patient_id, predictions in patient_predictions.items():
    
    binary_predictions = (np.array(predictions) > 0.5).astype(int)  

    
    patient_prediction = 0 if np.sum(binary_predictions == 0) > len(binary_predictions) / 2 else 1

    
    final_patient_predictions.append(patient_prediction)
    true_labels.append(0)  




patient_accuracy = accuracy_score(true_labels, final_patient_predictions)
print(f"Patient-based Accuracy: {patient_accuracy:.4f}")

Patient-based Accuracy: 0.8005
